In [15]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.base.model import GenericLikelihoodModel
import numpy as np
from scipy.stats import norm

In [16]:
# Load the data with low_memory=False to avoid dtype warning
data = pd.read_csv("D:\\CHRIST\\Boot camp\\DATA\\NSSO68.csv", low_memory=False)

# View the first few rows and columns
print(data.head())
print(data.columns)

   slno       grp  Round_Centre  FSU_number  Round  Schedule_Number  Sample  \
0     1  4.10E+31             1       41000     68               10       1   
1     2  4.10E+31             1       41000     68               10       1   
2     3  4.10E+31             1       41000     68               10       1   
3     4  4.10E+31             1       41000     68               10       1   
4     5  4.10E+31             1       41000     68               10       1   

   Sector  state  State_Region  ...  pickle_v  sauce_jam_v  Othrprocessed_v  \
0       2     24           242  ...       0.0          0.0              0.0   
1       2     24           242  ...       0.0          0.0              0.0   
2       2     24           242  ...       0.0          0.0              0.0   
3       2     24           242  ...       0.0          0.0              0.0   
4       2     24           242  ...       0.0          0.0              0.0   

   Beveragestotal_v  foodtotal_v  foodtotal_q  sta

In [17]:
# Create a binary indicator for non-vegetarian status
data['non_veg'] = ((data['nonvegtotal_q'] > 0) | 
                   (data['eggsno_q'] > 0) | 
                   (data['fishprawn_q'] > 0) | 
                   (data['goatmeat_q'] > 0) | 
                   (data['beef_q'] > 0) | 
                   (data['pork_q'] > 0) | 
                   (data['chicken_q'] > 0) | 
                   (data['othrbirds_q'] > 0)).astype(int)

In [18]:
# Ensure the columns 'Age', 'MPCE_URP', and 'Education' are present
print(data[['Age', 'MPCE_URP', 'Education']].head())

   Age  MPCE_URP  Education
0   50   3304.80        8.0
1   40   7613.00       12.0
2   45   3461.40        7.0
3   75   3339.00        6.0
4   30   2604.25        7.0


In [19]:
# Drop rows with missing values in these columns
data.dropna(subset=['Age', 'MPCE_URP', 'Education', 'non_veg'], inplace=True)

In [20]:
# Prepare the data for the model
X = data[['Age', 'MPCE_URP', 'Education']]
y = data['non_veg']

# Add a constant to the model (intercept)
X = sm.add_constant(X)

In [21]:
# Custom Tobit model
class Tobit(GenericLikelihoodModel):
    def __init__(self, endog, exog, left=None, right=None, **kwds):
        self.left = left
        self.right = right
        super(Tobit, self).__init__(endog, exog, **kwds)

    def nloglikeobs(self, params):
        exog = self.exog
        endog = self.endog
        beta = params[:-1]
        sigma = params[-1]
        xb = np.dot(exog, beta)
        z_left = (self.left - xb) / sigma if self.left is not None else None
        z_right = (self.right - xb) / sigma if self.right is not None else None

        ll = np.where(endog < self.left, np.log(norm.cdf(z_left)),
                      np.where(endog > self.right, np.log(norm.sf(z_right)),
                               norm.logpdf((endog - xb) / sigma) - np.log(sigma)))
        return -ll

    def fit(self, start_params=None, maxiter=10000, maxfun=5000, **kwds):
        if start_params is None:
            start_params = np.append(np.zeros(self.exog.shape[1]), 1)
        return super(Tobit, self).fit(start_params=start_params, maxiter=maxiter, maxfun=maxfun, **kwds)

# Fit the Tobit model
tobit_model = Tobit(y, X, left=0, right=1).fit()

# Summary of the model
print(tobit_model.summary())

C:\Users\isrin\AppData\Local\Temp\ipykernel_11324\223527810.py:17: RuntimeWarning: divide by zero encountered in log
  ll = np.where(endog < self.left, np.log(norm.cdf(z_left)),
C:\Users\isrin\AppData\Local\Temp\ipykernel_11324\223527810.py:18: RuntimeWarning: divide by zero encountered in log
  np.where(endog > self.right, np.log(norm.sf(z_right)),


Optimization terminated successfully.
         Current function value: 0.718814
         Iterations: 212
         Function evaluations: 352
                                Tobit Results                                 
Dep. Variable:                non_veg   Log-Likelihood:                -73071.
Model:                          Tobit   AIC:                         1.462e+05
Method:            Maximum Likelihood   BIC:                         1.462e+05
Date:                Mon, 01 Jul 2024                                         
Time:                        20:10:50                                         
No. Observations:              101655                                         
Df Residuals:                  101651                                         
Df Model:                           3                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------

D:\CHRIST\PYTHON\Lib\site-packages\statsmodels\base\model.py:2742: UserWarning: df_model + k_constant + k_extra differs from k_params
  warnings.warn("df_model + k_constant + k_extra "
D:\CHRIST\PYTHON\Lib\site-packages\statsmodels\base\model.py:2746: UserWarning: df_resid differs from nobs - k_params
  warnings.warn("df_resid differs from nobs - k_params")
